<a href="https://colab.research.google.com/github/Tushitaa18/ChaosLock/blob/main/K49_JAPANESE_CNN_ESN_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras import layers
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential, Model
from sklearn.model_selection import train_test_split

Mounted at /content/gdrive


In [ ]:
!pip install bayesian-optimization

In [ ]:
# Paths
drive_path = '/content/gdrive/MyDrive/Colab Notebooks/japanese_datset/k49'
train_imgs_path = os.path.join(drive_path, 'k49-train-imgs.npz')
train_labels_path = os.path.join(drive_path, 'k49-train-labels.npz')
test_imgs_path = os.path.join(drive_path, 'k49-test-imgs.npz')
test_labels_path = os.path.join(drive_path, 'k49-test-labels.npz')




In [ ]:
# Data loading
X_train = np.load(train_imgs_path)['arr_0']
Y_train= np.load(train_labels_path)['arr_0']
X_test= np.load(test_imgs_path)['arr_0']
Y_test= np.load(test_labels_path)['arr_0']

In [ ]:

X_train = X_train.reshape(232365,28,28,1)

X_test = X_test.reshape(38547,28,28,1)


In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.10)

In [ ]:
Y_train = keras.utils.to_categorical(Y_train)
Y_test= keras.utils.to_categorical(Y_test)
Y_val=keras.utils.to_categorical(Y_val)
Y_test.shape

(38547, 49)

In [ ]:

new_model=keras.models.load_model(os.path.join(drive_path,"CNN_model.h5"))

In [ ]:
# Let's look at our model
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 13, 13, 32)        128       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 64)          3

In [ ]:
def adjust_dimensions(val, target_length):
    if val is not None:
        val = np.array(val)
        if val.ndim == 0:
            val = np.array([val] * target_length)
        elif val.ndim == 1:
            if not len(val) == target_length:
                raise ValueError("Argument must have length " + str(target_length))
        else:
            raise ValueError("Invalid argument")
    return val

def identity_func(x):
    return x

In [ ]:
loss, acc = new_model.evaluate(X_test, Y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

1205/1205 - 14s - loss: 0.2250 - accuracy: 0.9428 - 14s/epoch - 11ms/step
Restored model, accuracy: 94.28%


In [ ]:
def adjust_dimensions(val, target_length):
    if val is not None:
        val = np.array(val)
        if val.ndim == 0:
            val = np.array([val] * target_length)
        elif val.ndim == 1:
            if not len(val) == target_length:
                raise ValueError("Argument must have length " + str(target_length))
        else:
            raise ValueError("Invalid argument")
    return val

def identity_func(x):
    return x

In [ ]:
import numpy as np
from keras.datasets import mnist
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from scipy.special import softmax
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.decomposition import PCA
from bayes_opt import BayesianOptimization

In [ ]:
class MyEchoStateNetwork():
    def __init__(self, num_inputs, num_outputs, reservoir_size=200,
                 spectral_radius=0.95, sparsity=0, noise=0.001, input_shift=None,
                 input_scaling=None, teacher_forcing=True, feedback_scaling=None,
                 teacher_scaling=None, teacher_shift=None,
                 output_activation=identity_func, inverse_output_activation=identity_func,
                 random_state=None, verbose=False):
        self.num_inputs = num_inputs
        self.reservoir_size = reservoir_size
        self.num_outputs = num_outputs
        self.spectral_radius = spectral_radius
        self.sparsity = sparsity
        self.noise = noise
        self.input_shift = adjust_dimensions(input_shift, num_inputs)
        self.input_scaling = adjust_dimensions(input_scaling, num_inputs)
        self.teacher_scaling = teacher_scaling
        self.teacher_shift = teacher_shift
        self.feedback_scaling = feedback_scaling  # Add this line
        self.output_activation = output_activation
        self.inverse_output_activation = inverse_output_activation
        self.random_state = random_state

        if isinstance(random_state, np.random.RandomState):
            self.random_state_ = random_state
        elif random_state:
            try:
                self.random_state_ = np.random.RandomState(random_state)
            except TypeError as e:
                raise Exception("Invalid seed: " + str(e))
        else:
            self.random_state_ = np.random.mtrand._rand

        self.teacher_forcing = teacher_forcing
        self.verbose = verbose
        self.initialize_weights()

    def initialize_weights(self):
        W = self.random_state_.rand(self.reservoir_size, self.reservoir_size) - 0.5
        W[self.random_state_.rand(*W.shape) < self.sparsity] = 0
        radius = np.max(np.abs(np.linalg.eigvals(W)))
        self.W = W * (self.spectral_radius / radius)
        self.W_in = self.random_state_.rand(
            self.reservoir_size, self.num_inputs) * 2 - 1
        self.W_feedback = self.random_state_.rand(
            self.reservoir_size, self.num_outputs) * 2 - 1

    def update_state(self, state, input_pattern, output_pattern):
        if self.teacher_forcing:
            preactivation = (np.dot(self.W, state)
                             + np.dot(self.W_in, input_pattern)
                             + np.dot(self.W_feedback, output_pattern))
        else:
            preactivation = (np.dot(self.W, state)
                             + np.dot(self.W_in, input_pattern))
        return (np.tanh(preactivation)
                + self.noise * (self.random_state_.rand(self.reservoir_size) - 0.5))

    def scale_inputs(self, inputs):
        if self.input_scaling is not None:
            inputs = np.dot(inputs, np.diag(self.input_scaling))
        if self.input_shift is not None:
            inputs = inputs + self.input_shift
        return inputs

    def scale_teacher(self, teacher):
        if self.teacher_scaling is not None:
            teacher = teacher * self.teacher_scaling
        if self.teacher_shift is not None:
            teacher = teacher + self.teacher_shift
        return teacher

    def unscale_teacher(self, teacher_scaled):
        if self.teacher_shift is not None:
            teacher_scaled = teacher_scaled - self.teacher_shift
        if self.teacher_scaling is not None:
            teacher_scaled = teacher_scaled / self.teacher_scaling
        return teacher_scaled

    def fit(self, inputs, outputs):
        if inputs.ndim < 2:
            inputs = np.reshape(inputs, (len(inputs), -1))
        if outputs.ndim < 2:
            outputs = np.reshape(outputs, (len(outputs), -1))

        # One-hot encode the outputs
        encoder = OneHotEncoder(sparse=False)
        outputs = encoder.fit_transform(outputs.reshape(-1, 1))

        inputs_scaled = self.scale_inputs(inputs)
        teachers_scaled = self.scale_teacher(outputs)

        states = np.zeros((inputs.shape[0], self.reservoir_size))
        for n in range(1, inputs.shape[0]):
            states[n, :] = self.update_state(states[n - 1], inputs_scaled[n, :],
                                             teachers_scaled[n - 1, :])

        transient = min(int(inputs.shape[1] / 10), 100)
        extended_states = np.hstack((states, inputs_scaled))
        self.W_out = np.dot(np.linalg.pinv(extended_states[transient:, :]),
                            self.inverse_output_activation(teachers_scaled[transient:, :])).T

        return self

    def predict(self, inputs):
      if inputs.ndim < 2:
        inputs = np.reshape(inputs, (len(inputs), -1))

      n_samples = inputs.shape[0]

      last_state = np.zeros(self.reservoir_size)
      last_input = np.zeros(self.num_inputs)
      last_output = np.zeros(self.num_outputs)

      inputs = np.vstack([last_input, self.scale_inputs(inputs)])
      states = np.vstack([last_state, np.zeros((n_samples, self.reservoir_size))])
      outputs = np.vstack([last_output, np.zeros((n_samples, self.num_outputs))])

      for n in range(n_samples):
        states[n + 1, :] = self.update_state(states[n, :], inputs[n + 1, :], outputs[n, :])
        outputs[n + 1, :] = self.output_activation(np.dot(self.W_out, np.concatenate([states[n + 1, :], inputs[n + 1, :]])))

      # Apply softmax to the outputs and return the class with the highest probability
      probabilities = softmax(outputs[1:], axis=1)
      return np.argmax(probabilities, axis=1)

In [ ]:
cnn_feature_extractor = Model(inputs=new_model.input, outputs=new_model.layers[-2].output)
X_train_features = cnn_feature_extractor.predict(X_train)
X_test_features = cnn_feature_extractor.predict(X_test)

1205/1205 [==============================] - 11s 9ms/step


In [ ]:
y_pred = new_model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(Y_test, axis=None)

1205/1205 [==============================] - 10s 8ms/step


In [ ]:
num_inputs = X_train_features.shape[1]
num_outputs = 49  # Number of classes in MNIST
X_train_features = X_train_features.reshape((X_train_features.shape[0], -1))
X_test_features = X_test_features.reshape((X_test_features.shape[0], -1))
X_train_features.shape
Y_train.shape

(209128, 49)

In [ ]:
Y_train = np.argmax(Y_train, axis=1)

In [ ]:
esn = MyEchoStateNetwork(num_inputs=num_inputs, num_outputs=49, reservoir_size=1000, spectral_radius=0.95, sparsity=0.05, noise=0.001, teacher_forcing=True, output_activation=identity_func, inverse_output_activation=identity_func, random_state=42, verbose=False)

In [ ]:
esn.fit(X_train_features, Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
y_pred_esn = esn.predict(X_test_features)
y_pred_esn.shape

(38547,)

In [ ]:
Y_test.shape

(38547,)

In [ ]:
y_pred_labels_esn = y_pred_esn
y_test_labels=Y_test
# y_test_labels=np.argmax(Y_test, axis=1)
# Y_test = np.argmax(Y_test, axis=1)

In [ ]:
accuracy_esn = accuracy_score(y_test_labels, np.array(y_pred_labels_esn).reshape(y_test_labels.shape))

In [ ]:
print(accuracy_esn)

0.9240667237398501


In [ ]:
def train_and_evaluate_esn(reservoir_size, spectral_radius, sparsity, noise):
    esn = MyEchoStateNetwork(num_inputs=num_inputs, num_outputs=49, reservoir_size=400, spectral_radius=0.16, sparsity=0.49, noise=0.06, teacher_forcing=True, output_activation=identity_func, inverse_output_activation=identity_func, random_state=42, verbose=False)

    esn.fit(X_train_features, Y_train)
    y_pred_esn = esn.predict(X_test_features)
   # y_pred_labels_esn = np.argmax(y_pred_esn, axis=1)
    y_pred_labels_esn = y_pred_esn
    # y_pred_labels_esn = np.reshape(y_pred_esn, (-1, 1))

    y_test_labels=Y_test

    accuracy_esn = accuracy_score(y_test_labels, np.array(y_pred_labels_esn).reshape(y_test_labels.shape))
    return accuracy_esn

In [ ]:
# Bayesian Optimization for hyperparameter tuning
pbounds = {'reservoir_size': (0, 800),
           'spectral_radius': (0, 1.0),
           'sparsity': (0.0, 1.0),
           'noise': (0.000001, 0.1)}

In [ ]:
optimizer = BayesianOptimization(f=train_and_evaluate_esn, pbounds=pbounds, random_state=42)
optimizer.maximize(init_points=10, n_iter=50)

|   iter    |  target   |   noise   | reserv... | sparsity  | spectr... |
-------------------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


| 1         | 0.9107    | 0.03745   | 760.6     | 0.732     | 0.5987    |


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


| 2         | 0.9107    | 0.0156    | 124.8     | 0.05808   | 0.8662    |


...

| 42        | 0.9107    | 0.02632   | 0.1354    | 0.1477    | 0.7234    |


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


| 43        | 0.9107    | 0.06005   | 139.0     | 0.5737    | 0.785     |


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


KeyboardInterrupt: ignored